<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/test/embedding%E6%89%B9%E9%87%8F%E5%92%8C%E6%89%93%E5%8C%85%E6%96%B0%E8%A7%92%E8%89%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/LC1332/Zero-Haruhi
%cd /content/Zero-Haruhi

/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 337 (delta 214), reused 251 (delta 151), pack-reused 3
Receiving objects: 100% (337/337), 2.85 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (214/214), done.
/content/Zero-Haruhi


In [3]:
def get_response( message ):
    return "语言模型输出了角色扮演的结果"

from ChatHaruhi import ChatHaruhi
chatbot = ChatHaruhi( role_name = "女贤者", role_from_hf = 'silk-road/ChatHaruhi-Waifu/女贤者', llm = get_response, max_story_n_haruhi = 2)
chatbot.add_rag_prompt_after_persona()

message = chatbot.get_message( user = "用户", text = "你好" )

Generating train split: 0 examples [00:00, ? examples/s]

query_rags: [{'n': 2, 'max_token': 1000, 'query': '你好', 'lid': 6}] rest_limit = 1660


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

In [4]:
roles = [
    "Dumbledore.jsonl", "Harry.jsonl", "Hermione.jsonl", "Luna.jsonl", "Malfoy.jsonl",
    "McGonagall.jsonl", "Penny.jsonl", "Raj.jsonl", "Ron.jsonl", "Sheldon.jsonl",
    "Snape.jsonl", "ayaka.jsonl", "baizhantang.jsonl", "duanyu.jsonl", "guofurong.jsonl",
    "haruhi.jsonl", "hutao.jsonl", "jiumozhi.jsonl", "linghuchong.jsonl", "liyunlong.jsonl",
    "murongfu.jsonl", "qiaofeng.jsonl", "raidenShogun.jsonl", "tangshiye.jsonl",
    "tongxiangyu.jsonl", "wanderer.jsonl", "wangduoyu.jsonl", "wangyuyan.jsonl",
    "weixiaobao.jsonl", "xiaofeng.jsonl", "xinghui.jsonl", "xuzhu.jsonl", "yuebuqun.jsonl",
    "yuqian.jsonl", "zhongli.jsonl"
]

In [5]:
%mkdir /content/output

In [24]:
model_name = "BAAI/bge-small-zh-v1.5"
save_embed_name = "bge_zh_s15"
save_folder = "/content/output"


from datasets import load_dataset
from ChatHaruhi import ChatHaruhi
from ChatHaruhi.embeddings import get_general_embeddings_safe
from ChatHaruhi.utils import float_array_to_base64
from ChatHaruhi.utils import save_datas_to_jsonl

import copy

# dataset = load_dataset("silk-road/ChatHaruhi-RolePlaying")
for role in roles:
    dataset = load_dataset("silk-road/ChatHaruhi-RolePlaying",data_files={'train':role})
    if 'bge_zh_s15' in dataset['train'][0].keys():
        continue
    role_from_hf = "silk-road/ChatHaruhi-RolePlaying" + "/" + role[:-6]
    chatbot = ChatHaruhi(role_name = "any", role_from_hf = role_from_hf, embed_name = 'openai', llm = get_response)

    save_name = save_folder + "/" + role


    stories = chatbot.db.stories

    vecs = get_general_embeddings_safe( stories, model_name = model_name )

    save_datas = []

    persona = chatbot.persona

    embed_name = save_embed_name

    # save_datas.append({ 'text':persona , embed_name:'system_prompt'})
    # save_datas.append({ 'text':'Reserve Config Setting Here' , embed_name:'config'})

    # keys = dataset['train'][0]

    vecs = ["system_prompt","config"] + vecs

    for data, vec in zip(dataset['train'], vecs):

        save_data = copy.deepcopy(data)
        if isinstance( vec, str ):
            save_data[embed_name] = vec
        else:
            save_data[embed_name] = float_array_to_base64(vec)
        save_datas.append(save_data)

    save_datas_to_jsonl(save_name, save_datas)
    # break

100%|██████████| 14/14 [00:01<00:00, 13.25it/s]


Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████| 88/88 [00:07<00:00, 11.68it/s]


Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████| 49/49 [00:04<00:00, 11.78it/s]


...

100%|██████████| 11/11 [00:01<00:00, 10.25it/s]


Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████| 108/108 [00:04<00:00, 23.04it/s]


Generating train split: 0 examples [00:00, ? examples/s]

100%|██████████| 17/17 [00:01<00:00, 15.99it/s]


对于/output/下的文件，进行压缩，保存在/content/updated_jsonl.zip，告诉我如何用python实现

In [25]:
import os
import zipfile

# 设置源目录和目标ZIP文件的路径
source_dir = '/content/output/'
target_zip = '/content/updated_jsonl.zip'

# 创建一个ZipFile对象，以写模式打开
with zipfile.ZipFile(target_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # os.walk遍历source_dir目录
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            # 创建文件的完整路径
            file_path = os.path.join(root, file)
            # 计算在ZIP文件中的路径
            in_zip_path = file_path.replace(source_dir, '')
            # 向ZIP文件中添加文件
            zipf.write(file_path, in_zip_path)

print(f"Files from '{source_dir}' have been zipped to '{target_zip}'")


Files from '/content/output/' have been zipped to '/content/updated_jsonl.zip'
